In [49]:
import getpass
import sys
import time

import numpy as np
from copy import deepcopy

from utils import calculate_perplexity, get_ptb_dataset, Vocab
from utils import ptb_iterator, sample

import tensorflow as tf
from tensorflow.python.ops.seq2seq import sequence_loss, sequence_loss_by_example
# from tensorflow.contrib.seq2seq.python.ops.loss import sequence_loss_by_example
# from tensorflow.models.rnn import seq2seq
# import tensorflow.nn.seq2seq
# from tensorflow.contrib.legacy_seq2seq.python.ops.seq2seq import sequence_loss, sequence_loss_by_example
from model import LanguageModel

ImportError: No module named legacy_seq2seq.python.ops.seq2seq

In [2]:
def load_data(debug=False):
    """Loads starter word-vectors and train/dev/test data."""
    vocab = Vocab()
    vocab.construct(get_ptb_dataset('train'))
    encoded_train = np.array(
        [vocab.encode(word) for word in get_ptb_dataset('train')],
        dtype=np.int32)
    encoded_valid = np.array(
        [vocab.encode(word) for word in get_ptb_dataset('valid')],
        dtype=np.int32)
    encoded_test = np.array(
        [vocab.encode(word) for word in get_ptb_dataset('test')],
        dtype=np.int32)
    if debug:
      num_debug = 1024
      encoded_train = encoded_train[:num_debug]
      encoded_valid = encoded_valid[:num_debug]
      encoded_test = encoded_test[:num_debug]

In [38]:
vocab = Vocab()
vocab.construct(get_ptb_dataset('train'))
len(vocab)

929589.0 total words with 10000 uniques


10000

In [11]:
vocab.index_to_word

{0: '<unk>',
 1: 'aer',
 2: 'banknote',
 3: 'berlitz',
 4: 'calloway',
 5: 'centrust',
 6: 'cluett',
 7: 'fromstein',
 8: 'gitano',
 9: 'guterman',
 10: 'hydro-quebec',
 11: 'ipo',
 12: 'kia',
 13: 'memotec',
 14: 'mlx',
 15: 'nahb',
 16: 'punts',
 17: 'rake',
 18: 'regatta',
 19: 'rubens',
 20: 'sim',
 21: 'snack-food',
 22: 'ssangyong',
 23: 'swapo',
 24: 'wachter',
 25: '<eos>',
 26: 'pierre',
 27: 'N',
 28: 'years',
 29: 'old',
 30: 'will',
 31: 'join',
 32: 'the',
 33: 'board',
 34: 'as',
 35: 'a',
 36: 'nonexecutive',
 37: 'director',
 38: 'nov.',
 39: 'mr.',
 40: 'is',
 41: 'chairman',
 42: 'of',
 43: 'n.v.',
 44: 'dutch',
 45: 'publishing',
 46: 'group',
 47: 'rudolph',
 48: 'and',
 49: 'former',
 50: 'consolidated',
 51: 'gold',
 52: 'fields',
 53: 'plc',
 54: 'was',
 55: 'named',
 56: 'this',
 57: 'british',
 58: 'industrial',
 59: 'conglomerate',
 60: 'form',
 61: 'asbestos',
 62: 'once',
 63: 'used',
 64: 'to',
 65: 'make',
 66: 'kent',
 67: 'cigarette',
 68: 'filters',
 69

In [12]:
vocab.word_to_index['aer']

1

In [17]:
encoded_train = np.array(
        [vocab.encode(word) for word in get_ptb_dataset('train')],
        dtype=np.int32)
encoded_valid = np.array(
        [vocab.encode(word) for word in get_ptb_dataset('valid')],
        dtype=np.int32)

In [19]:
print encoded_train.shape
print encoded_valid.shape

(929589,)
(73760,)


In [20]:
encoded_train

array([ 1,  2,  3, ..., 39,  0, 25], dtype=int32)

In [35]:
a = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]])
b = np.array([[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]], [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]])
b.shape

(2, 4, 4)

In [36]:
id = np.array([[1, 2, 3],[1, 2, 3],[1, 2, 3]])
m = np.array([1, 2, 3, 4])

In [30]:
l = tf.nn.embedding_lookup(a, id)

In [58]:
sess = tf.InteractiveSession()

In [34]:
l.eval()

array([[[ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]],

       [[ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]],

       [[ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]]])

In [54]:
b

array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]],

       [[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]]])

In [50]:
b1 = b.T

In [51]:
b2 = b1.T

In [53]:
print b.shape
print b1.shape
print b2.shape

(2, 4, 4)
(4, 4, 2)
(2, 4, 4)


In [58]:
np.reshape(b, (4, 2, 4))

array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8]],

       [[ 9, 10, 11, 12],
        [13, 14, 15, 16]],

       [[ 1,  2,  3,  4],
        [ 5,  6,  7,  8]],

       [[ 9, 10, 11, 12],
        [13, 14, 15, 16]]])

In [63]:
for ip in xrange(1, 10):
    print ip

1
2
3
4
5
6
7
8
9


In [99]:
e = b[0]
print e

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]


In [90]:
l = tf.split(1, 4, e)

In [113]:
tf.squeeze(l)

<tf.Tensor 'Squeeze_5:0' shape=(4, 4) dtype=int64>

In [70]:
v = tf.get_variable("v", initializer = tf.ones([1, 2]))

In [84]:
x = tf.zeros((2, 3))
print x.get_shape()


(2, 3)


In [81]:
x.eval()

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.]], dtype=float32)

In [118]:
for i in range(x.get_shape()[0]):
    print i

0
1


In [122]:
from tensorflow.python.ops.seq2seq import sequence_loss

In [97]:
output = tf.ones([64, 10000])
ops = [output, output, output]
labels_placeholder = tf.ones([64, 1])
labels_placeholder = tf.to_int32(labels_placeholder)
lb = [labels_placeholder, labels_placeholder, labels_placeholder]
weights = tf.ones([64, 1])
we = [weights, weights, weights]
# weights = tf.to_int32(weights)
print type(ops)

<type 'list'>


In [98]:
print output
print labels_placeholder
print weights

print len(ops)
print len(lb)
print len(we)

Tensor("ones_72:0", shape=(64, 10000), dtype=float32)
Tensor("ToInt32_25:0", shape=(64, 1), dtype=int32)
Tensor("ones_74:0", shape=(64, 1), dtype=float32)
3
3
3


In [96]:
# loss = sequence_loss([output], [labels_placeholder], [weights])
loss = sequence_loss(ops, lb, we)

ValueError: Tensor conversion requested dtype float32 for Tensor with dtype int32: 'Tensor("ToInt32_24:0", shape=(64, 1), dtype=int32)'

In [93]:
loss.eval()

589.46295